In [2]:
import pandas as pd
import numpy as np

# processed dataset
df = pd.read_csv("processed_data.csv")

<ipython-input-2-df3cf428c368>:5: DtypeWarning: Columns (12,13,14,15,16,17,18,19,20,21,22,24,25,26,27,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("processed_data.csv")


In [3]:
# Load file
import pandas as pd

file_path = 'processed_data.csv'
processed_data = pd.read_csv(file_path)

# Display the columns
print(processed_data.columns)

columns_list = list(processed_data.columns)
print(columns_list)

<ipython-input-3-03f69bd48188>:5: DtypeWarning: Columns (12,13,14,15,16,17,18,19,20,21,22,24,25,26,27,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  processed_data = pd.read_csv(file_path)


Index(['FL_DATE', 'CRS_DEP_TIME', 'OP_UNIQUE_CARRIER', 'DEST',
       'wind_dir_degrees', 'wind_speed_kt', 'wind_gust_kt',
       'visibility_statute_mi', 'temperature_c', 'dewpoint_c', 'altimeter_hpa',
       'Flight_Status', 'Carrier_AA', 'Carrier_AS', 'Carrier_B6', 'Carrier_DL',
       'Carrier_F9', 'Carrier_G4', 'Carrier_HA', 'Carrier_NK', 'Carrier_OO',
       'Carrier_UA', 'Carrier_WN', 'DEST_REGION', 'Dest_Region_Midwest',
       'Dest_Region_Mountain', 'Dest_Region_Northeast', 'Dest_Region_OCONUS',
       'Dest_Region_South', 'Dest_Region_Southwest', 'Dest_Region_West',
       'Flight_Status_Binary'],
      dtype='object')
['FL_DATE', 'CRS_DEP_TIME', 'OP_UNIQUE_CARRIER', 'DEST', 'wind_dir_degrees', 'wind_speed_kt', 'wind_gust_kt', 'visibility_statute_mi', 'temperature_c', 'dewpoint_c', 'altimeter_hpa', 'Flight_Status', 'Carrier_AA', 'Carrier_AS', 'Carrier_B6', 'Carrier_DL', 'Carrier_F9', 'Carrier_G4', 'Carrier_HA', 'Carrier_NK', 'Carrier_OO', 'Carrier_UA', 'Carrier_WN', 'DEST_RE

### Training, Validation, Test Sets

# **Split Data 70/15/15**

##Balancing the Dataset (Undersampling)

In [8]:
from sklearn.utils import resample

#  features and target
train_data = pd.concat([X_train, y_train], axis=1)

# Separate the majority and minority classes
majority_class = train_data[train_data[target] == 0]
minority_class = train_data[train_data[target] == 1]

# Undersample the majority class
majority_class_downsampled = resample(majority_class,
                                      replace=False,
                                      n_samples=len(minority_class),
                                      random_state=42)

# Combine the downsampled majority class with the minority class
balanced_train_data = pd.concat([majority_class_downsampled, minority_class])

# Separate back into X and y
X_train_balanced = balanced_train_data.drop(columns=[target])
y_train_balanced = balanced_train_data[target]

# Check
print(f"Class distribution in training set after balancing:\n{y_train_balanced.value_counts()}")

Class distribution in training set after balancing:
Flight_Status_Binary
0.0    13592
1.0    13592
Name: count, dtype: int64


In [4]:
# Feature selection
features_to_use = [
    'wind_dir_degrees', 'wind_speed_kt', 'temperature_c', 'dewpoint_c',
    'visibility_statute_mi', 'altimeter_hpa', 'Carrier_AA', 'Carrier_AS',
    'Carrier_B6', 'Carrier_DL', 'Carrier_F9', 'Carrier_G4', 'Carrier_HA',
    'Carrier_NK', 'Carrier_OO', 'Carrier_UA', 'Carrier_WN', 'Dest_Region_Midwest',
    'Dest_Region_Mountain', 'Dest_Region_Northeast', 'Dest_Region_OCONUS',
    'Dest_Region_South', 'Dest_Region_Southwest', 'Dest_Region_West'
]

# target variable
target = 'Flight_Status_Binary'

In [16]:
# Drop rows with NaN in the target column
processed_data = processed_data.dropna(subset=[target])

# Now split again
X = processed_data[features_to_use]
y = processed_data[target]

# Split into training and temp (validation + test)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Split temp into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

In [15]:
from sklearn.model_selection import train_test_split

# Split into features (X) and target (y)
X = processed_data[features_to_use]
y = processed_data[target]

# Split into training and temp (validation + test) datasets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Split temp into validation and test sets (50% for validation, 50% for test)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Check the sizes of the splits
print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Test set size: {len(X_test)}")

Training set size: 106749
Validation set size: 22875
Test set size: 22875


##Feature Scaling

In [9]:
from sklearn.preprocessing import StandardScaler

# StandardScaler
scaler = StandardScaler()

# transform the training data
X_train_scaled = scaler.fit_transform(X_train_balanced)

# Transform the validation and test data
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Check
print(f"Scaled Training data shape: {X_train_scaled.shape}")
print(f"Scaled Validation data shape: {X_val_scaled.shape}")
print(f"Scaled Test data shape: {X_test_scaled.shape}")

Scaled Training data shape: (27184, 24)
Scaled Validation data shape: (22875, 24)
Scaled Test data shape: (22875, 24)


## Model Training (RandomForest) for 70/15/15 split

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# RandomForest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model using the scaled training data
rf_model.fit(X_train_scaled, y_train_balanced)

# Make predictions on the validation set
val_preds = rf_model.predict(X_val_scaled)

# Evaluate performance on the validation set
print("Validation Set Performance:")
print(confusion_matrix(y_val, val_preds))
print(classification_report(y_val, val_preds))

Validation Set Performance:
[[12764  7198]
 [ 1092  1821]]
              precision    recall  f1-score   support

         0.0       0.92      0.64      0.75     19962
         1.0       0.20      0.63      0.31      2913

    accuracy                           0.64     22875
   macro avg       0.56      0.63      0.53     22875
weighted avg       0.83      0.64      0.70     22875



In [11]:
# Make predictions on the test set
test_preds = rf_model.predict(X_test_scaled)

# Evaluate performance on the test set
print("\nTest Set Performance:")
print(confusion_matrix(y_test, test_preds))
print(classification_report(y_test, test_preds))


Test Set Performance:
[[12703  7260]
 [ 1049  1863]]
              precision    recall  f1-score   support

         0.0       0.92      0.64      0.75     19963
         1.0       0.20      0.64      0.31      2912

    accuracy                           0.64     22875
   macro avg       0.56      0.64      0.53     22875
weighted avg       0.83      0.64      0.70     22875



# Results for 70/15/15 Split
Validation Set:

The model was good at predicting on-time flights (94% precision), but it had trouble predicting delayed/canceled flights (only 18% precision).

It correctly identified 64% of all the delayed/canceled flights, but missed a lot of them.

Test Set:

The results were similar to the validation set: the model was good at predicting on-time flights (94% precision), but struggled with delayed/canceled flights (18% precision).

It correctly identified 64% of delayed/canceled flights, but still missed many.

# **80/10/10 split**

##Since our goal number 2 is to reduce delay times by informing logistics, its better to catch more delays. Using a lower threshold to increase recall, since missing a delay would be worse than predicting one that doesn't end up happening

In [12]:
from sklearn.model_selection import train_test_split

# Split the data into 80% training, 10% validation, and 10% testing
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Split the remaining data (X_temp, y_temp) into 50% validation and 50% test (i.e., 10% each)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Check the split sizes
print("Training set size:", len(X_train))
print("Validation set size:", len(X_val))
print("Test set size:", len(X_test))

Training set size: 121999
Validation set size: 15250
Test set size: 15250


In [19]:
# Step 1: Drop rows where the target is missing
df_cleaned = df.dropna(subset=['Flight_Status_Binary'])

# Step 2: First split (80% train, 20% temp)
train_df_80, temp_df = train_test_split(
    df_cleaned, test_size=0.20, random_state=42, stratify=df_cleaned['Flight_Status_Binary']
)

# Step 3: Split the 20% temp into 10% val, 10% test
val_df_10, test_df_10 = train_test_split(
    temp_df, test_size=0.50, random_state=42, stratify=temp_df['Flight_Status_Binary']
)

# Confirm sizes
print("Train size:", len(train_df_80))
print("Validation size:", len(val_df_10))
print("Test size:", len(test_df_10))

Train size: 93888
Validation size: 11736
Test size: 11736


## Logistic Regression for 80/10/10 split


In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import resample

# Features used for modeling
features = [
    'wind_dir_degrees', 'wind_speed_kt', 'temperature_c', 'dewpoint_c',
    'visibility_statute_mi', 'altimeter_hpa',
    'Carrier_AA', 'Carrier_AS', 'Carrier_B6', 'Carrier_DL', 'Carrier_F9',
    'Carrier_G4', 'Carrier_HA', 'Carrier_NK', 'Carrier_OO', 'Carrier_UA', 'Carrier_WN',
    'Dest_Region_Midwest', 'Dest_Region_Mountain', 'Dest_Region_Northeast',
    'Dest_Region_OCONUS', 'Dest_Region_South', 'Dest_Region_Southwest', 'Dest_Region_West'
]

# Split X and y
X_train = train_df_80[features]
y_train = train_df_80['Flight_Status_Binary']

X_val = val_df_10[features]
y_val = val_df_10['Flight_Status_Binary']

X_test = test_df_10[features]
y_test = test_df_10['Flight_Status_Binary']

# 🔁 Undersample the majority class in training data
majority = train_df_80[train_df_80['Flight_Status_Binary'] == 0]
minority = train_df_80[train_df_80['Flight_Status_Binary'] == 1]

majority_downsampled = resample(
    majority, replace=False, n_samples=len(minority), random_state=42
)

balanced_train_df = pd.concat([majority_downsampled, minority]).sample(frac=1, random_state=42)

X_train_bal = balanced_train_df[features]
y_train_bal = balanced_train_df['Flight_Status_Binary']

# 📏 Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_bal)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# 🚀 Train logistic regression
logreg = LogisticRegression(max_iter=1000, random_state=42)
logreg.fit(X_train_scaled, y_train_bal)

# 🔍 Evaluate on validation set
val_preds = logreg.predict(X_val_scaled)
print("🟦 Validation Set Performance:")
print(confusion_matrix(y_val, val_preds))
print(classification_report(y_val, val_preds))

# 🔍 Evaluate on test set
test_preds = logreg.predict(X_test_scaled)
print("🟩 Test Set Performance:")
print(confusion_matrix(y_test, test_preds))
print(classification_report(y_test, test_preds))

🟦 Validation Set Performance:
[[5607 4224]
 [ 750 1155]]
              precision    recall  f1-score   support

         0.0       0.88      0.57      0.69      9831
         1.0       0.21      0.61      0.32      1905

    accuracy                           0.58     11736
   macro avg       0.55      0.59      0.50     11736
weighted avg       0.77      0.58      0.63     11736

🟩 Test Set Performance:
[[5593 4238]
 [ 762 1143]]
              precision    recall  f1-score   support

         0.0       0.88      0.57      0.69      9831
         1.0       0.21      0.60      0.31      1905

    accuracy                           0.57     11736
   macro avg       0.55      0.58      0.50     11736
weighted avg       0.77      0.57      0.63     11736



## Conclusion for Logistic 80/10/10 model

The logistic regression model achieved a recall of 0.60 on the test set, meaning it correctly identified 60% of delayed or canceled flights. However, its precision was lower at 0.21, meaning many on-time flights were incorrectly flagged. With an F1 score of 0.31, it shows a balanced but modest performance. This model is helpful for catching delays early, but it comes with more false alarms—something to consider when prioritizing precision vs. recall.


##XGBoost with 80/10/10 Split

In [30]:
# Ensure all features are numeric
for df in [train_df_80, val_df_10, test_df_10]:
    df[features] = df[features].apply(pd.to_numeric, errors='coerce')

In [34]:
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix

# Step 1: Define features and target
features = [
    'wind_dir_degrees', 'wind_speed_kt', 'wind_gust_kt', 'visibility_statute_mi',
    'temperature_c', 'dewpoint_c', 'altimeter_hpa',
    'Carrier_AA', 'Carrier_AS', 'Carrier_B6', 'Carrier_DL', 'Carrier_F9', 'Carrier_G4',
    'Carrier_HA', 'Carrier_NK', 'Carrier_OO', 'Carrier_UA', 'Carrier_WN',
    'Dest_Region_Midwest', 'Dest_Region_Mountain', 'Dest_Region_Northeast',
    'Dest_Region_OCONUS', 'Dest_Region_South', 'Dest_Region_Southwest', 'Dest_Region_West'
]

target = 'Flight_Status_Binary'

In [36]:
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# Step 2: Train the XGBoost model on the training set
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train_80.astype(float), y_train_80)

# Step 3: Predict probabilities for validation and test sets
val_probs = xgb_model.predict_proba(X_val_10.astype(float))[:, 1]
test_probs = xgb_model.predict_proba(X_test_10.astype(float))[:, 1]

# Step 4: Apply custom threshold at 20
custom_threshold = 0.20
val_preds_custom = (val_probs >= custom_threshold).astype(int)
test_preds_custom = (test_probs >= custom_threshold).astype(int)

# Step 4: Evaluate performance
print(f"\n Validation Confusion Matrix (Threshold = {custom_threshold}):")
print(confusion_matrix(y_val_10, val_preds_custom))
print(f"Precision: {precision_score(y_val_10, val_preds_custom):.2f}")
print(f"Recall: {recall_score(y_val_10, val_preds_custom):.2f}")
print(f"F1 Score: {f1_score(y_val_10, val_preds_custom):.2f}")

print(f"\n Test Confusion Matrix (Threshold = {custom_threshold}):")
print(confusion_matrix(y_test_10, test_preds_custom))
print(f"Precision: {precision_score(y_test_10, test_preds_custom):.2f}")
print(f"Recall: {recall_score(y_test_10, test_preds_custom):.2f}")
print(f"F1 Score: {f1_score(y_test_10, test_preds_custom):.2f}")

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [23:34:13] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



 Validation Confusion Matrix (Threshold = 0.2):
[[7678 2153]
 [ 901 1004]]
Precision: 0.32
Recall: 0.53
F1 Score: 0.40

 Test Confusion Matrix (Threshold = 0.2):
[[7712 2119]
 [ 934  971]]
Precision: 0.31
Recall: 0.51
F1 Score: 0.39


## Conclusion for XGBoost with .20 threshold (80/10/10 split)
Our XGBoost model with the 80/10/10 split and 0.2 threshold is helping us meet our project goals. It catches over half of the delayed or canceled flights (recall = 0.51) and gives a balanced performance (F1 score = 0.39). This means we can better predict delays and help teams act early to reduce them.

#Random Forest 80/10/10 Split

In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# Train the Random Forest model
rf_model_80 = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_80.fit(X_train_80, y_train_80)

# Predict probabilities
val_probs_rf = rf_model_80.predict_proba(X_val_10)[:, 1]
test_probs_rf = rf_model_80.predict_proba(X_test_10)[:, 1]

# Apply custom threshold
rf_threshold = 0.2
val_preds_rf_custom = (val_probs_rf >= rf_threshold).astype(int)
test_preds_rf_custom = (test_probs_rf >= rf_threshold).astype(int)

# Evaluate on validation set
print("✅ Validation Confusion Matrix (Threshold = 0.2):")
print(confusion_matrix(y_val_10, val_preds_rf_custom))
print("Precision:", round(precision_score(y_val_10, val_preds_rf_custom), 2))
print("Recall:", round(recall_score(y_val_10, val_preds_rf_custom), 2))
print("F1 Score:", round(f1_score(y_val_10, val_preds_rf_custom), 2))

# Evaluate on test set
print("\n✅ Test Confusion Matrix (Threshold = 0.2):")
print(confusion_matrix(y_test_10, test_preds_rf_custom))
print("Precision:", round(precision_score(y_test_10, test_preds_rf_custom), 2))
print("Recall:", round(recall_score(y_test_10, test_preds_rf_custom), 2))
print("F1 Score:", round(f1_score(y_test_10, test_preds_rf_custom), 2))

✅ Validation Confusion Matrix (Threshold = 0.2):
[[7434 2397]
 [ 871 1034]]
Precision: 0.3
Recall: 0.54
F1 Score: 0.39

✅ Test Confusion Matrix (Threshold = 0.2):
[[7486 2345]
 [ 950  955]]
Precision: 0.29
Recall: 0.5
F1 Score: 0.37


##Conclusion for Random forest

With a threshold of 0.2, the Random Forest model achieved a precision of 0.30, recall of 0.54, and F1 score of 0.39 on the validation set. On the test set, it had a precision of 0.29, recall of 0.50, and F1 score of 0.37. This means the model is doing a better job identifying delayed flights (good recall), though the trade-off is more false positives. It’s a solid step toward meeting our goal of early delay detection for better operational planning

#Summary of Models

We can use XGBoost if we want the best balance between catching delays (recall) and avoiding false alarms (precision).

Logistic regression is good for catching more delays (highest recall), but it flags more on-time flights incorrectly.

Random Forest is strong, but slightly behind XGBoost in precision and F1 score.

### Best Results?
XGBoost had the best F1 score and highest precision, which means its balanced and identifies delays while ALSO minimizing false positives.

In [39]:
import pandas as pd

# model results
model_comparison = pd.DataFrame({
    'Model': ['Random Forest', 'Logistic Regression', 'XGBoost'],
    'Precision': [0.29, 0.21, 0.31],
    'Recall': [0.50, 0.60, 0.51],
    'F1 Score': [0.37, 0.31, 0.39]
})

# table
model_comparison.sort_values(by='F1 Score', ascending=False).reset_index(drop=True)

,Model,Precision,Recall,F1 Score
0,XGBoost,0.31,0.51,0.39
1,Random Forest,0.29,0.50,0.37
2,Logistic Regression,0.21,0.60,0.31
